In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv("./pure-methods-languagetool.csv")
df.shape

(1876, 6)

In [3]:
def extract_from_tags(tag, tags):
    search_string = tag + "=[\w]+"
    extracted_tag = re.findall(search_string, tags)[0]
    extracted_tag = re.findall("\=(.*)", extracted_tag)[0]
    if extracted_tag == "true":
        return True
    else:
        return False

In [4]:
cols = ["visibility", "parent-FQN", "method-name", "param-list", "return-type", 
        "local-variables", "conditionals", "multiple-statements", "loops", "parameters", "returns",
        "switches", "ifs", "static", "returns-primitives"]
final_df = pd.DataFrame(columns = cols)
for index, row in df.iterrows():
    final_df.loc[index, 'visibility'] = row['visibility']
    final_df.loc[index, 'parent-FQN'] = row['parent-FQN']
    final_df.loc[index, 'method-name'] = row['method-name']
    final_df.loc[index, 'param-list'] = row['param-list'].strip('[]')
    final_df.loc[index, 'return-type'] = row['return-type']
    final_df.loc[index, 'local-variables'] = extract_from_tags("local_variables", str(row['tags']))
    final_df.loc[index, 'conditionals'] = extract_from_tags("conditionals", str(row['tags']))
    final_df.loc[index, 'multiple-statements'] = extract_from_tags("multiple_statements", str(row['tags']))
    final_df.loc[index, 'loops'] = extract_from_tags("loops", str(row['tags']))
    final_df.loc[index, 'parameters'] = extract_from_tags("parameters", str(row['tags']))
    final_df.loc[index, 'returns'] = extract_from_tags("returns", str(row['tags']))
    final_df.loc[index, 'switches'] = extract_from_tags("switches", str(row['tags']))
    final_df.loc[index, 'ifs'] = extract_from_tags("ifs", str(row['tags']))
    final_df.loc[index, 'static'] = extract_from_tags("static", str(row['tags']))
    final_df.loc[index, 'returns-primitives'] = extract_from_tags("returns_primitives", str(row['tags']))

final_df.head()

,visibility,parent-FQN,method-name,param-list,return-type,local-variables,conditionals,multiple-statements,loops,parameters,returns,switches,ifs,static,returns-primitives
0,public,org.languagetool.gui.Configuration,getUseDocLanguage,,boolean,False,False,False,False,False,True,False,False,False,True
1,public,org.languagetool.languagemodel.bert.grpc.BertL...,getResponsesOrBuilderList,,java.util.List,False,False,False,False,False,True,False,False,False,False
2,public,org.languagetool.rules.Category,getLocation,,org.languagetool.rules.Category$Location,False,False,False,False,False,True,False,False,False,False
3,public,org.languagetool.rules.patterns.AbstractPatter...,getFilterArguments,,java.lang.String,False,False,False,False,False,True,False,False,False,False
4,NaN,org.languagetool.dev.index.Searcher$SearchRunn...,getLuceneMatchCount,,int,False,False,False,False,False,True,False,False,False,True


### Methods that return objects

In [5]:
(final_df["returns-primitives"] == False).sum()

1505

### Methods that accept parameters

In [6]:
(final_df["parameters"] == True).sum()

94

### Methods with if statements

In [7]:
(final_df["ifs"] == True).sum()

22

### Methods wtih conditionals ?:

In [8]:
(final_df["conditionals"] == True).sum()

13

### Methods with local variables

In [9]:
(final_df["local-variables"] == True).sum()

9

### Methods with multiple statements

In [10]:
(final_df["multiple-statements"] == True).sum()

13

In [11]:
instrumentation_candidates_df = pd.DataFrame(columns = cols)
instrumentation_candidates_df = final_df[((final_df['multiple-statements'] == True) | 
                                          (final_df['ifs'] == True) |
                                          (final_df['conditionals'] == True) | 
                                          (final_df['parameters'] == True) | 
                                          (final_df['switches'] == True) |
                                          (final_df['loops'] == True) |
                                          (final_df['local-variables'] == True)) &
                                          (final_df['static'] == False) & 
                                          (final_df['visibility'] == "public")]
print(instrumentation_candidates_df.shape)
instrumentation_candidates_df.to_csv(r'./instrumentation-candidates-pdfbox.csv', index=False)
instrumentation_candidates_df.tail(60)

(55, 15)


,visibility,parent-FQN,method-name,param-list,return-type,local-variables,conditionals,multiple-statements,loops,parameters,returns,switches,ifs,static,returns-primitives
12,public,org.languagetool.tokenizers.el.GreekWordTokeni...,yycharat,int,char,False,False,False,False,True,True,False,False,False,True
29,public,org.languagetool.AnalyzedTokenReadings,getAnalyzedToken,int,org.languagetool.AnalyzedToken,False,False,False,False,True,True,False,False,False,False
52,public,org.languagetool.openoffice.MultiDocumentsHandler,getNumMinToCheckParas,,java.util.List,False,False,True,False,False,True,False,True,False,False
68,public,org.languagetool.rules.RuleMatch,getShortMessage,,java.lang.String,False,False,True,False,False,True,False,True,False,False
221,public,org.languagetool.openoffice.MultiDocumentsHandler,getServiceDisplayName,com.sun.star.lang.Locale,java.lang.String,False,False,False,False,True,True,False,False,False,False
245,public,org.languagetool.rules.RemoteRuleConfig,getDownMilliseconds,,long,False,True,False,False,False,True,False,False,False,True
390,public,org.languagetool.rules.patterns.PatternRuleId,toString,,java.lang.String,False,False,False,False,False,True,False,True,False,False
402,public,org.languagetool.rules.neuralnetwork.Suggestion,toString,,java.lang.String,False,False,False,False,False,True,False,True,False,False
418,public,org.languagetool.rules.patterns.AbstractPatter...,toString,,java.lang.String,False,True,False,False,False,True,False,False,False,False
435,public,org.languagetool.rules.ConfusionPair,toString,,java.lang.String,False,True,False,False,False,True,False,False,False,False


In [12]:
sorted(instrumentation_candidates_df['parent-FQN'].unique())

['org.languagetool.AnalyzedSentence',
 'org.languagetool.AnalyzedTokenReadings',
 'org.languagetool.Language',
 'org.languagetool.LinguServices',
 'org.languagetool.gui.Configuration',
 'org.languagetool.gui.RetainLineBreakTransferHandler',
 'org.languagetool.gui.RetainLineBreakTransferHandler$MyTransferable',
 'org.languagetool.noop.NoopDisambiguator',
 'org.languagetool.openoffice.MultiDocumentsHandler',
 'org.languagetool.rules.ConfusionPair',
 'org.languagetool.rules.ReadabilityRule',
 'org.languagetool.rules.RemoteRuleConfig',
 'org.languagetool.rules.RuleMatch',
 'org.languagetool.rules.WordRepeatRule',
 'org.languagetool.rules.ca.ReplaceOperationNamesRule',
 'org.languagetool.rules.ca.SimpleReplaceAnglicism',
 'org.languagetool.rules.ca.SimpleReplaceBalearicRule',
 'org.languagetool.rules.ca.SimpleReplaceDNVColloquialRule',
 'org.languagetool.rules.ca.SimpleReplaceDNVRule',
 'org.languagetool.rules.ca.SimpleReplaceDNVSecondaryRule',
 'org.languagetool.rules.ca.SimpleReplaceDiacr